In [1]:

# Install necessary libraries
%pip install "pybamm[plot,cite]" -q

import pybamm
import matplotlib.pyplot as plt

# Define the battery model
model = pybamm.lithium_ion.DFN(
    {
        "SEI": "solvent-diffusion limited",
        "SEI porosity change": "true",
        "lithium plating": "partially reversible",
        "lithium plating porosity change": "true",  # alias for "SEI porosity change"
        "particle mechanics": ("swelling and cracking", "swelling only"),
        "SEI on cracks": "true",
        "loss of active material": "stress-driven",
        "calculate discharge energy": "true",  # for compatibility with older PyBaMM versions
    }
)

# Set model parameters
param = pybamm.ParameterValues("OKane2022")
param.update({
    "SEI solvent diffusivity [m2.s-1]": 2e-21,  # The new value can be adjusted as needed
    'Dead lithium decay constant [s-1]': 7.5e-5,
    'Negative electrode cracking rate': 1.17e-18,
    'Positive electrode cracking rate': 1.17e-18,
})

var_pts = {
    "x_n": 5,  # Negative electrode
    "x_s": 5,  # Separator
    "x_p": 5,  # Positive electrode
    "r_n": 30,  # Negative electrode particles
    "r_p": 30,  # Positive electrode particles
}

# Set the target number of cycles
target_cycle = 900

print(f"Calculating cycle {target_cycle} ...")

# Define the experimental protocol
exp_k = pybamm.Experiment(
    [
        (
            "Discharge at 0.3C until 2.5 V",
            "Charge at 0.3C until 4.2 V (5 minute period)",
            "Hold at 4.2 V until C/100 (5 minute period)",
        )
    ]
    * target_cycle
    + ["Discharge at 0.3C until 2.5 V (5 minute period)"],
)

# Create solver with a smaller dt_max
solver = pybamm.CasadiSolver(dt_max=100)  # Try reducing further, e.g., dt_max=50 or 10 if needed
print(f"Current dt_max: {solver.dt_max}")

# Run the simulation
sim_k = pybamm.Simulation(model, parameter_values=param, experiment=exp_k, var_pts=var_pts, solver=solver)
sol_k = sim_k.solve()

# Extract LLI loss (unit: A.h) and calculate the remaining capacity
Q_final_k = sol_k['Discharge capacity [A.h]'].entries[-1]
remaining_capacity_k = Q_final_k / 5 * 100

print(f"When k={target_cycle}, the remaining capacity is: {remaining_capacity_k:.2f}%")

Note: you may need to restart the kernel to use updated packages.
Calculating cycle 900 ...
Current dt_max: 100


At t = 266.584 and h = 5.71076e-15, the corrector convergence failed repeatedly or with |h| = hmin.
The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
At t = 270.193 and h = 1.83409e-14, the corrector convergence failed repeatedly or with |h| = hmin.
The linesearch algorithm failed with too small a step.
At t = 214.888 and h = 5.04573e-14, the corrector convergence failed repeatedly or with |h| = hmin.
The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
At t = 251.868 and h = 1.21673e-13, the 

When k=900, the remaining capacity is: 39.91%
